pip install openpyxl


In [1]:
import pandas as pd

In [2]:
excel_file = '../data/clean_data/lapd_ripa_raw_data_08312021_mod.xlsx'

In [3]:
raw_data = pd.read_excel(excel_file, sheet_name=None)

In [4]:
raw_data.keys()

dict_keys(['MAIN', 'Officer', 'ACTIONS TAKEN', 'BASIS FOR SEARCH', 'Reason For Stop', 'Result of Stop', 'Person', 'Perceived Race', 'Perceived Disability', 'Person Contraband Discovered ', 'Result of Stop and Race'])

In [5]:
for sheet in raw_data.keys():
    if "Stop ID" in raw_data[sheet].columns:
        print("{} \t\t{} rows  \t {} unique stops ".format(sheet, len(raw_data[sheet]), len(raw_data[sheet]["Stop ID"].unique())))

MAIN 		1425 rows  	 1425 unique stops 
ACTIONS TAKEN 		1663 rows  	 1425 unique stops 
BASIS FOR SEARCH 		1468 rows  	 1425 unique stops 
Reason For Stop 		1447 rows  	 1425 unique stops 
Result of Stop 		1519 rows  	 1425 unique stops 
Person 		1440 rows  	 1425 unique stops 
Perceived Race 		1462 rows  	 1425 unique stops 
Perceived Disability 		1443 rows  	 1425 unique stops 
Person Contraband Discovered  		1449 rows  	 1425 unique stops 
Result of Stop and Race 		1573 rows  	 1425 unique stops 


In [16]:
data = raw_data['MAIN']
sheets_to_merge=['ACTIONS TAKEN', 'BASIS FOR SEARCH', 'Reason For Stop', 
                 'Result of Stop', 'Person', 'Perceived Race', 
                 'Perceived Disability', 'Person Contraband Discovered ']
for sheet in sheets_to_merge:
    data = data.merge(raw_data[sheet], how="inner", on="Stop ID")

In [17]:
data.keys()

Index(['Stop ID', 'Stop Date Time', 'Stop Duration',
       'Stop In Response To Calls for Service', 'Street',
       'Action Taken During Stop', 'Person Search Consent Given',
       'Property Search Consent Given', 'Basis For Search', 'Reason For Stop',
       'Result Of Stop', 'Perceived Limited English', 'Perceived Age',
       'Perceived Gender', 'Gender Nonconforming', 'Perceived LGBT',
       'Perceived Race', 'Perceived Or Known Disability',
       'Contraband Or Evidence Discovered'],
      dtype='object')

In [18]:
data

,Stop ID,Stop Date Time,Stop Duration,Stop In Response To Calls for Service,Street,Action Taken During Stop,Person Search Consent Given,Property Search Consent Given,Basis For Search,Reason For Stop,Result Of Stop,Perceived Limited English,Perceived Age,Perceived Gender,Gender Nonconforming,Perceived LGBT,Perceived Race,Perceived Or Known Disability,Contraband Or Evidence Discovered
0,55,2021-02-01 10:40:33.020,10,No,Foothill Expressway,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,50,Male,No,No,White,None,None
1,56,2021-02-01 10:54:23.067,5,No,San Antonio,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),Yes,25,Female,No,No,Hispanic/Latino/a,None,None
2,57,2021-02-01 17:27:29.437,15,No,foothill exp,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,25,Male,No,No,Hispanic/Latino/a,None,None
3,58,2021-02-01 19:21:04.627,10,No,Plaza Central,No Action,No,No,No Search,Traffic Violation,Citation for infraction,No,45,Female,No,No,White,None,None
4,59,2021-02-01 19:27:34.940,6,No,SAN ANTONIO RD,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,20,Female,No,No,White,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16172,2327,2021-07-30 22:06:19.767,5,No,San Antonio,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,18,Female,No,No,White,None,None
16173,2328,2021-07-31 15:46:58.243,8,No,Alvarado,No Action,No,No,No Search,Traffic Violation,Contacted parent/legal guardian or other perso...,No,17,Male,No,No,White,None,None
16174,2328,2021-07-31 15:46:58.243,8,No,Alvarado,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,17,Male,No,No,White,None,None
16175,2329,2021-07-31 23:28:24.150,3,No,GRANT RD,No Action,No,No,No Search,Traffic Violation,Warning (verbal or written),No,20,Female,No,No,White,None,None


In [19]:
len(data)

16177

In [20]:
len(data["Stop ID"].unique())

1425

In [22]:
data[data["Stop ID"] == 85].drop_duplicates()

,Stop ID,Stop Date Time,Stop Duration,Stop In Response To Calls for Service,Street,Action Taken During Stop,Person Search Consent Given,Property Search Consent Given,Basis For Search,Reason For Stop,Result Of Stop,Perceived Limited English,Perceived Age,Perceived Gender,Gender Nonconforming,Perceived LGBT,Perceived Race,Perceived Or Known Disability,Contraband Or Evidence Discovered
32,85,2021-02-05 15:38:50.533,25,No,Grant Road,Asked for consent to search property,No,No,No Search,Traffic Violation,In-field cite and release,No,35,Male,No,No,Hispanic/Latino/a,None,None
33,85,2021-02-05 15:38:50.533,25,No,Grant Road,Asked for consent to search property,No,No,No Search,Traffic Violation,Custodial Arrest pursuant to outstanding warrant,No,35,Male,No,No,Hispanic/Latino/a,None,None
34,85,2021-02-05 15:38:50.533,25,No,Grant Road,Person removed from vehicle by order,No,No,No Search,Traffic Violation,In-field cite and release,No,35,Male,No,No,Hispanic/Latino/a,None,None
35,85,2021-02-05 15:38:50.533,25,No,Grant Road,Person removed from vehicle by order,No,No,No Search,Traffic Violation,Custodial Arrest pursuant to outstanding warrant,No,35,Male,No,No,Hispanic/Latino/a,None,None
36,85,2021-02-05 15:38:50.533,25,No,Grant Road,Asked for consent to search person,Yes,No,No Search,Traffic Violation,In-field cite and release,No,35,Male,No,No,Hispanic/Latino/a,None,None
37,85,2021-02-05 15:38:50.533,25,No,Grant Road,Asked for consent to search person,Yes,No,No Search,Traffic Violation,Custodial Arrest pursuant to outstanding warrant,No,35,Male,No,No,Hispanic/Latino/a,None,None


In [28]:
data["Result Of Stop"].value_counts()

Warning (verbal or written)                                                  8222
No Action                                                                    3499
Citation for infraction                                                      3186
In-field cite and release                                                     592
Custodial Arrest pursuant to outstanding warrant                              363
Custodial Arrest without warrant                                              241
Field interview card completed                                                 32
Contacted parent/legal guardian or other person responsible for the minor      15
Psychiatric hold                                                               15
Noncriminal transport or caretaking transport                                  12
Name: Result Of Stop, dtype: int64

In [30]:
data["Stop ID"].value_counts()

1555    6561
1123    2560
1313     768
1157     768
1432     512
        ... 
1794       1
1786       1
1778       1
1762       1
2047       1
Name: Stop ID, Length: 1425, dtype: int64

In [31]:
data.drop_duplicates()["Stop ID"].value_counts()

2217    72
1176    48
1313    32
1432    32
2215    24
        ..
1606     1
1604     1
1602     1
1600     1
2047     1
Name: Stop ID, Length: 1425, dtype: int64

In [32]:
data[data["Stop ID"] == 2217]

,Stop ID,Stop Date Time,Stop Duration,Stop In Response To Calls for Service,Street,Action Taken During Stop,Person Search Consent Given,Property Search Consent Given,Basis For Search,Reason For Stop,Result Of Stop,Perceived Limited English,Perceived Age,Perceived Gender,Gender Nonconforming,Perceived LGBT,Perceived Race,Perceived Or Known Disability,Contraband Or Evidence Discovered
15973,2217,2021-07-18 01:21:18.450,189,No,El Monte,Curbside detention,No,No,Evidence of crime,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Drug Paraphernalia
15974,2217,2021-07-18 01:21:18.450,189,No,El Monte,Curbside detention,No,No,Evidence of crime,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Other Contraband or evidence
15975,2217,2021-07-18 01:21:18.450,189,No,El Monte,Curbside detention,No,No,Evidence of crime,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Suspected Stolen property
15976,2217,2021-07-18 01:21:18.450,189,No,El Monte,Curbside detention,No,No,Evidence of crime,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Drugs/narcotics
15977,2217,2021-07-18 01:21:18.450,189,No,El Monte,Curbside detention,No,No,Incident to arrest,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Drug Paraphernalia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16040,2217,2021-07-18 01:21:18.450,189,No,El Monte,Search of property was conducted,No,No,Incident to arrest,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Drugs/narcotics
16041,2217,2021-07-18 01:21:18.450,189,No,El Monte,Search of property was conducted,No,No,Vehicle inventory,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Drug Paraphernalia
16042,2217,2021-07-18 01:21:18.450,189,No,El Monte,Search of property was conducted,No,No,Vehicle inventory,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Other Contraband or evidence
16043,2217,2021-07-18 01:21:18.450,189,No,El Monte,Search of property was conducted,No,No,Vehicle inventory,Traffic Violation,Custodial Arrest without warrant,No,30,Female,No,No,Hispanic/Latino/a,None,Suspected Stolen property
